In [ ]:
import torch
from torch import optim

def mpc_control(state, target_state, horizon, dt, m=0.2, M=1.0, L=0.5, g=9.81):
    """
    Model Predictive Controller for the linearized inverted pendulum.
    """
    # Initialize the control inputs (horizon steps)
    u = torch.zeros(horizon, 1, requires_grad=True)  # Shape: (horizon, 1)
    
    # Optimizer to minimize cost
    optimizer = optim.Adam([u], lr=0.1)

    # MPC optimization loop
    for _ in range(100):  # Number of optimization iterations
        state_pred = state.clone()
        cost = 0.0

        # Predict states over the horizon and compute cost
        for t in range(horizon):
            state_pred = simulate_dynamics(state_pred, u[t].unsqueeze(0), dt, m, M, L, g)
            # Cost function (e.g., minimize position and angle error)
            cost += ((state_pred - target_state) ** 2).sum()

        # Perform optimization step
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

    # Return the first control input (MPC applies only the first input and recalculates at each step)
    return u[0].detach()